# YOLOv8

In [2]:
import os
import argparse
import logging
import matplotlib.pyplot as plt
from ultralytics import YOLO
from IPython.display import display, Image

## Download Images from Bucket

In [ ]:
import sys
sys.path.append('../src')
from data_utils import download_images_and_labels


# Bucket and directory
bucket_name = 'tree-counting-project'
prefix = 'Dataset 348-17-15/' # using Roboflow dataset with 348 train, 17 val and 15 test images
local_dir = '../Data/Dataset 348-17-15'

# Download images
download_images_and_labels(bucket_name, prefix, local_dir)

## Installing YOLO

In [1]:
!pip install ultralytics==8.0.196

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.196 🚀 Python-3.10.14 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 14918MiB)
Setup complete ✅ (1 CPUs, 3.6 GB RAM, 41.2/98.2 GB disk)


## Training

In [8]:
# Function for logging
def setup_logging(log_dir):
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
    logging.basicConfig(filename=os.path.join(log_dir, 'log.txt'),
                        filemode='a',
                        format='%(message)s',
                        level=logging.DEBUG)
    logger = logging.getLogger()
    return logger


In [13]:
# function to plot training and val loss curves
def plot_metrics(log_dir, metrics):
    epochs = range(len(metrics['train_loss']))
    
    plt.figure()
    plt.plot(epochs, metrics['train_loss'], label='Train Loss')
    plt.plot(epochs, metrics['val_loss'], label='Val Loss')
    plt.title('Loss Curves')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig(os.path.join(log_dir, 'loss_curve.png'))
    plt.close()
    
    plt.figure()
    plt.plot(epochs, metrics['train_map50'], label='Train mAP50')
    plt.plot(epochs, metrics['val_map50'], label='Val mAP50')
    plt.title('mAP50 Curves')
    plt.xlabel('Epochs')
    plt.ylabel('mAP50')
    plt.legend()
    plt.savefig(os.path.join(log_dir, 'map50_curve.png'))
    plt.close()


In [9]:
def train_yolo(model_name, data_path, total_epochs, img_size, log_dir):
    # Load model
    model = YOLO(model_name)

    # Initialize metrics dictionary
    metrics = {'train_loss': [], 'val_loss': [], 'train_map50': [], 'val_map50': []}
    
    # Setup logging
    logger = setup_logging(log_dir)

    for epoch in range(total_epochs):
        # Train model for one epoch
        results = model.train(
            data=os.path.join(data_path, 'data.yaml'),
            epochs=1,
            imgsz=img_size,
            plots=False,
            save=True
        )
        
        # Extract metrics
        train_loss = results.metrics['train']['loss'][0]  # assuming loss is stored in the first index
        val_loss = results.metrics['val']['loss'][0]  # assuming loss is stored in the first index
        train_map50 = results.metrics['train']['map50'][0]  # mAP at IoU=0.50 for training
        val_map50 = results.metrics['val']['map50'][0]  # mAP at IoU=0.50 for validation
        
        # Store metrics
        metrics['train_loss'].append(train_loss)
        metrics['val_loss'].append(val_loss)
        metrics['train_map50'].append(train_map50)
        metrics['val_map50'].append(val_map50)

        # Log metrics
        log_entry = {
            'epoch': epoch,
            'train_loss': train_loss,
            'val_loss': val_loss,
            'train_map50': train_map50,
            'val_map50': val_map50,
            'n_parameters': model.model.n_params()
        }
        logger.info(json.dumps(log_entry))
        
        # Save checkpoint
        model.save(f'{log_dir}/checkpoint_epoch_{epoch}.pt')

    # Plot metrics
    plot_metrics(log_dir, metrics)

In [15]:
# argument parsing
def parse_args():
    parser = argparse.ArgumentParser(description="Train YOLOv8 model.")
    parser.add_argument('--model', type=str, default='yolov8s.pt', help='Pre-trained model path')
    parser.add_argument('--data', type=str, required=True, help='Path to dataset')
    parser.add_argument('--epochs', type=int, default=25, help='Number of epochs to train')
    parser.add_argument('--img_size', type=int, default=800, help='Image size for training')
    parser.add_argument('--log_dir', type=str, default='./logs', help='Directory to save logs and plots')
    return parser.parse_args()

def parse_args_notebook():
    args = argparse.Namespace(
        model='yolov8s.pt',
        data='/path/to/your/dataset',
        epochs=25,
        img_size=800,
        log_dir='./logs'
    )
    return args


In [16]:
args = parse_args_notebook()

# Put in arguments for training
args.data = '/path/to/your/dataset'
args.epochs = 30
args.img_size = 640
args.log_dir = './training_logs'
